Import cleaned data from the local filepath

In [1]:
import argparse
import os
import shutil
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
# from get_embedding_function import get_embedding_function
from langchain.vectorstores.chroma import Chroma
from typing import List, Dict, Any

from langchain_community.embeddings.ollama import OllamaEmbeddings

CHROMA_PATH="chroma"

In [2]:
loader = CSVLoader(file_path='./data/cleaned_slang_data.csv')
slang_document = loader.load()

In [3]:
print(slang_document[0])
print(type(slang_document))
print(type(slang_document[0]))
len(slang_document)


page_content='word: Janky
definition: Undesirable; less-than optimum.' metadata={'source': './data/cleaned_slang_data.csv', 'row': 0}
<class 'list'>
<class 'langchain_core.documents.base.Document'>


2580653

In [4]:
# Only necessary if we have too much data to add to the context.
def split_documents(documents: list[Document]):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=80,
    length_function=len,
    is_separator_regex=False,
  )
  return text_splitter.split_documents(documents)

chunks = split_documents(slang_document)

In [5]:
print(chunks[0])
print(len(chunks))

page_content='word: Janky
definition: Undesirable; less-than optimum.' metadata={'source': './data/cleaned_slang_data.csv', 'row': 0}
2706525


Add it to chroma db

In [10]:

def get_embedding_function():
  embeddings = OllamaEmbeddings(model="llama3.1")
  return embeddings

In [11]:
if os.path.exists(CHROMA_PATH):
  shutil.rmtree(CHROMA_PATH)

def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
      persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )
    db.add_documents(chunks)
    db.persist()

add_to_chroma(chunks)

Number of existing documents in DB: 0
👉 Adding new documents: 2706525


KeyboardInterrupt: 